In [12]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [13]:
import Distill

In [14]:
from texts import text_2

In [15]:
text = Distill.remove_stopwords(Distill.tokenize(text_2))

In [16]:
text = Distill.remove_tags(text)

In [17]:
text = Distill.lemmatize(text)

In [18]:
# text = Distill._to_string(text)

In [22]:
tokens_ = []
for a in text:
    tokens_.append([a])

In [24]:
# tokens_

In [26]:
id2word = corpora.Dictionary(tokens_)

In [27]:
def to_token(List):
    # takes a simple list and breaks it into tokens of the form [[],[],[]]
    token = [[a] for a in List]
    return token

In [28]:
saas = to_token(text)

In [30]:
# saas

In [31]:
corpus = [id2word.doc2bow(text) for text in tokens_]

In [39]:
# Human readable format of corpus (term-frequency)
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:]]

In [40]:
# corpus[:]

In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [42]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.087*"property" + 0.046*"brake" + 0.046*"lead" + 0.004*"commonwealth" + '
  '0.004*"compensate" + 0.004*"design" + 0.004*"decade" + 0.004*"range" + '
  '0.004*"local" + 0.004*"mean"'),
 (1,
  '0.160*"damage" + 0.040*"reach" + 0.040*"expose" + 0.040*"contaminate" + '
  '0.004*"range" + 0.004*"assessment" + 0.004*"design" + 0.004*"public" + '
  '0.004*"mean" + 0.004*"reimbursement"'),
 (2,
  '0.196*"party" + 0.070*"private" + 0.037*"caustic" + 0.037*"action" + '
  '0.004*"mean" + 0.004*"design" + 0.004*"public" + 0.004*"assessment" + '
  '0.004*"commonwealth" + 0.004*"range"'),
 (3,
  '0.120*"habitat" + 0.044*"agree" + 0.044*"fabric" + 0.004*"party" + '
  '0.004*"compensate" + 0.004*"design" + 0.004*"public" + 0.004*"assessment" + '
  '0.004*"commonwealth" + 0.004*"encourage"'),
 (4,
  '0.313*"site" + 0.143*"ecological" + 0.052*"predecessor" + 0.003*"mean" + '
  '0.003*"fws" + 0.003*"range" + 0.003*"compensate" + 0.003*"commonwealth" + '
  '0.003*"reimbursement" + 0.003*"assessm

In [44]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens_, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -20.580518818099517

Coherence Score:  0.8399551092841173


In [45]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis